# IAM Credential Compromise - Containment
This notebook is to be used in case of malicious activity inside your AWS Account.
We will query CloudTrail logs using Athena in order to detect and contain malicious account activity.

We are following best practices from the AWS Incident Response guide as we execute our processes:

https://docs.aws.amazon.com/whitepapers/latest/aws-security-incident-response-guide/aws-security-incident-response-guide.html

# Setup

## Load Libraries

In order to query CloudTrail and interact with AWS, we need to load several libraries and configure our environment.

In [ ]:
pip install pyathena --quiet

In [ ]:
import boto3  # the Python SDK for AWS
import pandas as pd # Pandas is a data analysis tool for Python
from pyathena import connect # Python API client for Amazon Athena
region='us-east-1' # Set region variable to us-east-1 for API commands
athena_bucket = 's3:// ENTER BUCKET NAME HERE /'  # S3 bucket that is configured to store your Athena queries
db_name = 'security_analysis' # database used by Athena. Choose 'default' if your CloudTrail was created via the Athena console and 'security_analysis' if you are using the AWS Security Analytics Bootstrap

## 1.0 Set up helper function for Athena

The Python query_results function shown below will help you query Athena tables.

In [ ]:
def query_results(sql):
    
    cursor = connect(s3_staging_dir=athena_bucket, region_name=region).cursor()
    cursor.execute(sql)
    
    columns = cursor.description
    data = cursor.fetchall()

    column_names = [column[0] for column in columns]
    rows = [dict(zip(column_names, data_row)) for data_row in data]

    df = pd.DataFrame(rows, columns=column_names)
    df1 = df.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    
    return df1.set_properties(**{'text-align': 'center'})

## 3. IAM Containment
Now that you have identified the potential IAM entity which has been compromised you need to perform containment activities. The first of these will be to find out what the Access Key ID is being used by the account.


### 3.1 Identify Access Key ID

In [ ]:
keyid = f"""

select useridentity.accesskeyid, count(*) as total
from "{db_name}"."cloudtrail" 
where userIdentity.arn='insert ARN of IAM Credential from query in 2.2'
group by useridentity.accesskeyid
order by total desc

"""

In [ ]:
results = query_results(keyid)
results

### 3.2 Deactivate Access Key

In [ ]:
access_key_to_deactivate='ENTER ACCESS KEY ID HERE'
username='ENTER USERNAME HERE'
iam = boto3.resource('iam', region_name=region)
access_key = iam.AccessKey(username,access_key_to_deactivate)
response_status = access_key.deactivate()
status_code = response_status['ResponseMetadata']['HTTPStatusCode']
if status_code == 200:
    print('Key Disabled Successfully')
else:
    print('Key deactivation failed')

### 3.3 Attach Deny All Actions Policy

In [ ]:
username='ENTER USERNAME HERE'
iam = boto3.client('iam', region_name=region)
response = iam.put_user_policy(UserName=username,PolicyName='Block',PolicyDocument='{"Version":"2012-10-17","Statement":{"Effect":"Deny","Action":"*","Resource":"*"}}')
status_code = response['ResponseMetadata']['HTTPStatusCode']
if status_code == 200:
    print('Policy attached successfully')
else:
    print('Policy attachment failed')


## [Credential Compromise Detection Notebook](./credential-compromise-detection.ipynb)